TODO0: use LSTM (2 layers) similarly to Artexte for direct comparison 

TODO1: separate all this stuff to different files and stick to the default trainer and config

TODO2: use jsonnet config to pass dataset reader to both model and train command

TODO3: during backtranslation, first greedly backtranslate with torch.not_grad() 

TODO4: then convert output into list of instances into "Batch" into tensor_dict

TODO5: call model forward on this tensor_dict and get gradients

TODO6: then repeat for other languages (for loop), but reuse the same variables to save GPU 
memory

TODO7: finally refactor to get ready for transformers

TODO8: use fairseq transformer

# Main

In [2]:
from unsupervised_translation.data.dataset_readers.unsupervised_translation_reader import UnsupervisedTranslationDatasetsReader
from unsupervised_translation.models.unsupervised_translation import UnsupevisedTranslation
from unsupervised_translation.data.iterators.homogeneous_batch_iterator import HomogeneousBatchIterator

from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.token_embedders.embedding import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
import torch
import allennlp
from allennlp.training import Trainer

In [10]:
umt_dr = UnsupervisedTranslationDatasetsReader(lazy=True)

data_files = {"en": "fixtures/data/mono.en", "ru": "fixtures/data/mono.ru"}
data_files = {"en-et": "abracadabra", "en": "fixtures/data/mono.en", "ru": "fixtures/data/mono.ru"}

train_dataset = umt_dr.read(data_files)
validation_dataset = umt_dr.read(data_files)

vocab = Vocabulary.from_instances(train_dataset)

EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
text_field_embedding = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = UnsupevisedTranslation(vocab=vocab, source_embedder=text_field_embedding, encoder=lstm, max_decoding_steps=10, target_namespace="tokens")

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

iterator = HomogeneousBatchIterator(type_field_name = "lang", batch_size = 2)
iterator.index_with(vocab)
iterator.get_num_batches(train_dataset)

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=3)

12it [00:00, 2709.64it/s]


In [4]:
trainer.train()

loss: 4.2367 ||: : 7it [00:00, 63.02it/s]
BLEU: 0.0000, loss: 0.0000 ||: : 7it [00:00, 88.42it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

[['ochumla', 'my', 'my', 'my', 'my', 'my', 'my', 'There', 'my'], ['ochumla', 'my', 'my', 'my', 'my', 'There', 'There', 'There', 'There']]
[['my', 'my', 'my', 'my', 'my'], ['ochumla', 'my', 'my', 'my', 'my']]
[['my', 'my', 'my', 'my', 'my', 'my'], ['my', 'my', 'my', 'my', 'my', 'my']]
[['my', 'my', 'my', 'my', 'my', 'my'], ['my', 'my', 'my', 'my', 'my', 'my']]
[['my', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'There', 'There'], ['my', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my']]
[['my', 'my', 'my', 'my']]
[['ochumla', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my']]
[['ochumla', 'my', 'my', 'my', 'my', 'my', 'my', 'There', 'my'], ['ochumla', 'my', 'my', 'my', 'my', 'There', 'There', 'There', 'There']]


loss: 4.2076 ||: : 7it [00:00, 54.71it/s]
BLEU: 0.0000, loss: 0.0000 ||: : 7it [00:00, 82.85it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

[['my', 'my', 'my', 'my', 'my'], ['ochumla', 'my', 'my', 'my', 'my']]
[['my', 'my', 'my', 'my', 'my', 'my'], ['my', 'my', 'my', 'my', 'my', 'my']]
[['my', 'my', 'my', 'my', 'my', 'my'], ['my', 'my', 'my', 'my', 'my', 'my']]
[[',', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'There', 'There'], [',', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my']]
[[',', 'my', 'my', 'my']]
[['ochumla', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my']]
[[',', ',', 'my', 'my', 'my', 'my', 'my', 'my', 'my'], [',', 'my', 'my', 'my', 'my', 'There', 'There', 'There', 'There']]


loss: 4.1791 ||: : 7it [00:00, 67.15it/s]
BLEU: 0.0000, loss: 0.0000 ||: : 7it [00:00, 91.11it/s]

[[',', 'my', 'my', 'my', 'my'], [',', 'my', 'my', 'my', 'my']]
[[',', ',', 'my', 'my', 'my', 'my'], [',', ',', 'my', 'my', 'my', 'my']]
[[',', 'my', 'my', 'my', 'my', 'my'], [',', 'my', 'my', 'my', 'my', 'my']]
[[',', ',', 'my', 'my', 'my', 'my', 'my', 'my', 'There', 'There'], [',', ',', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my']]
[[',', 'my', 'my', 'my']]
[[',', 'my', 'my', 'my', 'my', 'my', 'my', 'my', 'my']]


{'best_epoch': 0,
 'peak_cpu_memory_MB': 252.516,
 'training_duration': '00:00:00',
 'training_start_epoch': 0,
 'training_epochs': 2,
 'epoch': 2,
 'training_loss': 4.179075990404401,
 'training_cpu_memory_MB': 252.516,
 'validation_BLEU': 1.7587783859894286e-12,
 'validation_loss': 0.0,
 'best_validation_BLEU': 1.7587783859894286e-12,
 'best_validation_loss': 0.0}

# Predict

In [5]:
from overrides import overrides

from allennlp.common.util import JsonDict
from allennlp.data import Instance
from allennlp.predictors.predictor import Predictor


class MySeq2SeqPredictor(Predictor):
    """
    Predictor for sequence to sequence models, including
    :class:`~allennlp.models.encoder_decoder.simple_seq2seq` and
    :class:`~allennlp.models.encoder_decoder.copynet_seq2seq`.
    """

    def predict(self, source: str, lang: str) -> JsonDict:
        return self.predict_json({"lang": lang, "source" : source})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        """
        Expects JSON that looks like ``{"lang", "source": "..."}``.
        """
        source = json_dict["source"]
        lang = json_dict["lang"]
        return self._dataset_reader.text_to_instance(source, lang)

In [6]:
predictor = MySeq2SeqPredictor(model, UnsupervisedTranslationDatasetsReader())

In [7]:
predictor.predict("There are two mouses in front of a cat.", "en")

{'class_log_probabilities': [-37.89716339111328],
 'predictions': [[40, 40, 40, 40, 40, 40, 40, 40, 40, 40]],
 'predicted_tokens': ['my',
  'my',
  'my',
  'my',
  'my',
  'my',
  'my',
  'my',
  'my',
  'my']}